In [ ]:
import chromadb
import os
from pyprojroot import here
from google import genai
from google.genai import types
import spacy
from dotenv import load_dotenv
from pprint import pprint
load_dotenv()

True

**Load environment variables and configs**

In [ ]:
EMBEDDING_MODEL = "en_core_web_lg"
VECTORDB_DIR = "data/airline_policy_vectordb"
K=2

**Load the vectorDB**

In [ ]:
client = chromadb.PersistentClient(path=str(here(VECTORDB_DIR)))
collection = client.get_or_create_collection(name="rag-chroma")
print("Number of vectors in vectordb:", collection.count(), "\n\n")

Number of vectors in vectordb: 30 




**Sample Query**

In [4]:
message = "What is the cancelation rule for a flight ticket at swiss airline policy?"

**Perform the vector Search**

In [ ]:
nlp = spacy.load(EMBEDDING_MODEL)
query_embedding = nlp(message).vector

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=K,
    include=["documents", "metadatas"]
)

In [ ]:
results

{'ids': [['8', '25']],
 'embeddings': None,
 'documents': [['If you need an invoice for personal or business purposes, most countries accept an e-ticket confirmation as an invoice.\nFor countries with special requirements, an invoice can be ordered using forms that are available for this purpose.\nUsing an e-ticket confirmation as an invoice Your e-ticket confirmation is internationally recognised as a receipt in most\ncountries, including Switzerland, and it can be used for the reimbursement of expenses and for the submission of a tax\nreturn.\nYou can order your e-ticket confirmation in your profile under “Bookings” via the link “E-documents (Invoice)” or via the\nlink “ Passenger Receipt”.\nOrdering an invoice after your booking If you find out at a later date that you require an invoice, we can send you a new\nbooking confirmation free of charge up to 90 days after the ticket/document has been completely used. After this 90-day\nperiod, we charge a fee of CHF 30 for each confirmati

**Prepare the prompt for the LLM model**

In [7]:
question = "# User new question:\n" + message
retrieved_content = ""
for doc in results["documents"][0]:
    retrieved_content += f"{doc}\n\n"
prompt = f"# Content:\n{retrieved_content}\n\n{question}"

Prepared prompt

In [8]:
pprint(prompt)

('# Content:\n'
 'If you need an invoice for personal or business purposes, most countries '
 'accept an e-ticket confirmation as an invoice.\n'
 'For countries with special requirements, an invoice can be ordered using '
 'forms that are available for this purpose.\n'
 'Using an e-ticket confirmation as an invoice Your e-ticket confirmation is '
 'internationally recognised as a receipt in most\n'
 'countries, including Switzerland, and it can be used for the reimbursement '
 'of expenses and for the submission of a tax\n'
 'return.\n'
 'You can order your e-ticket confirmation in your profile under “Bookings” '
 'via the link “E-documents (Invoice)” or via the\n'
 'link “ Passenger Receipt”.\n'
 'Ordering an invoice after your booking If you find out at a later date that '
 'you require an invoice, we can send you a new\n'
 'booking confirmation free of charge up to 90 days after the ticket/document '
 'has been completely used. After this 90-day\n'
 'period, we charge a fee of CHF 3

**Pass the prompt to the LLM model and get the response**

In [ ]:
client = genai.Client()
model = "gemini-2.5-flash"
response = client.models.generate_content(
        model=model,
        contents=prompt,
        config=types.GenerateContentConfig(
            thinking_config=types.ThinkingConfig(thinking_budget=0), # Disables thinking
            system_instruction="You will receive a user's query and possible content where the answer might be. If the answer is found, provide it, if not, state that the answer does not exist."
        ),
    )

Printing the response

In [ ]:
pprint(response.text)

('Swiss Airlines has a 24-hour cancellation policy that allows passengers to '
 'cancel their flights within 24 hours of booking without penalty. This policy '
 'applies to all fare types, including non-refundable tickets. If you cancel '
 "your Swiss Airlines flight within 24 hours of booking, you'll receive a full "
 'refund of your ticket price.\n'
 '\n'
 'Important things to keep in mind:\n'
 '* This policy only applies to flights booked directly through Swiss '
 'Airlines. If you booked through a travel agent or third-party website, you '
 'need to check their cancellation policy.\n'
 '* If you cancel your flight after the 24-hour window, you may be subject to '
 'cancellation fees or penalties.')


**RAG Tool design using LangChain**

In [ ]:
from langchain_core.tools import tool

@tool
def lookup_swiss_airline_policy(query: str)->str:
    """Search within the Swiss Airline's company policies to check whether certain options are permitted. Input should be a search query."""
    # Load spaCy model and compute embedding
    nlp = spacy.load(EMBEDDING_MODEL)
    query_embedding = nlp(query).vector

    # Load Chroma collection
    client = chromadb.PersistentClient(path=str(here(VECTORDB_DIR)))
    collection = client.get_or_create_collection(name="rag-chroma")

    # Query Chroma
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=K,
        include=["documents"]
    )

    # Combine results into a single string
    return "\n\n".join(results["documents"][0])

In [12]:
print(lookup_swiss_airline_policy.name)
print(lookup_swiss_airline_policy.args)
print(lookup_swiss_airline_policy.description)

lookup_swiss_airline_policy
{'query': {'title': 'Query', 'type': 'string'}}
Search within the Swiss Airline's company policies to check whether certain options are permitted. Input should be a search query.


In [13]:
pprint(lookup_swiss_airline_policy.invoke("can I cancel my ticket?"))

('hour cancellation policy that allows passengers to cancel their flights '
 'within 24 hours of booking at +1-877-507-7341\n'
 'without penalty. This policy applies to all fare types, including '
 'non-refundable tickets. If you cancel your Swiss Airlines\n'
 "flight within 24 hours of booking, you'll receive a full refund of your "
 'ticket price.\n'
 'How to Cancel Swiss Airlines Flight within 24 Hours? If you need to cancel '
 'your Swiss Airlines flight within 24 hours of\n'
 'booking, you can do so easily online. Here are the steps to follow:\n'
 'Go to Swiss Airlines\' website and click on the "Manage your bookings" tab. '
 'Enter your booking reference number and last\n'
 'name to access your booking. Select the flight you want to cancel and click '
 'on "Cancel flight." Confirm your cancellation\n'
 "and you'll receive a full refund of your ticket price. If you booked your "
 "Swiss Airlines flight through a travel agent, you'll\n"
 'need to contact them directly to cancel you